In [ ]:
from popsborder.simulation import run_simulation
from popsborder.scenarios import run_scenarios
from popsborder.inputs import load_configuration, load_scenario_table
from popsborder.outputs import save_scenario_result_to_pandas, print_totals_as_text

import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Set path to folder containing config and scenario files
datadir = os.path.join(os.getcwd(), "data")
# Set path to folder for saving results and automatically create folder if it doesn't exist
resultsdir = os.path.join(os.getcwd(), "results")
# Make sure the directory exists
if not os.path.exists(resultsdir):
    os.mkdir(resultsdir)

### Run a single simulation

In [ ]:
config_path = datadir + "/user_friendly_config.xlsx"

In [ ]:
num_simulations = 1
num_consignments = 5
config = load_configuration(config_path, sheet=None, key_column="D", value_column="B")
results = run_simulation(
    config,
    num_simulations,
    num_consignments,
    seed=42,
    output_f280_file=None,
    verbose=False,
    pretty="boxes",
    detailed=False,
)
print_totals_as_text(num_consignments, config, totals=results)

### Run multiple scenarios

In [ ]:
scenario_configs_path = datadir + "/scenarios_config.csv"

In [ ]:
num_consignments = 100
scenario_configs = load_configuration(config, sheet=None, key_column="D", value_column="B")
results = run_scenarios(
    config=scenario_configs,
    scenario_table=load_scenario_table(scenario_configs_path),
    seed=42,
    num_simulations=10,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
results_df = save_scenario_result_to_pandas(
    results,
    config_columns=[
        "name",
        "contamination/contamination_rate/value",
        "contamination/arrangement",
        "inspection/selection_strategy",
        "inspection/unit",
        "inspection/sample_strategy",
    ],
    result_columns=[
        "true_contamination_rate",
        "max_missed_contamination_rate",
        "avg_missed_contamination_rate",
        "max_intercepted_contamination_rate",
        "avg_intercepted_contamination_rate",
        "avg_boxes_opened_completion",
        "pct_boxes_opened_completion",
        "avg_boxes_opened_detection",
        "pct_boxes_opened_detection",
        "avg_items_inspected_completion",
        "pct_items_inspected_completion",
        "avg_items_inspected_detection",
        "pct_items_inspected_detection",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
        "num_boxes",
        "num_items",
    ],
)

In [ ]:
results_df['failure rate'] = results_df["intercepted"] / num_consignments
contaminated_consignments = results_df["false_neg"] + results_df["intercepted"]
results_df["interception rate"] = results_df["intercepted"] / contaminated_consignments
results_df["% missed contaminants"] = (results_df["total_missed_contaminants"] / (results_df["total_missed_contaminants"] + results_df["total_intercepted_contaminants"])) * 100

In [ ]:
results_df

In [ ]:
plt.barh(results_df["name"], results_df["% missed contaminants"])
plt.title("Missed contaminants", fontsize=18)
plt.ylabel("Inspection name", fontsize=14)
plt.xlabel("Pct. missed contaminants", fontsize=14)

In [ ]:
plt.barh(results_df["name"], results_df["pct_items_inspected_completion"]*100)
plt.title("Items inspected", fontsize=18)
plt.ylabel("Inspection name", fontsize=14)
plt.xlabel("Pct. items inspected", fontsize=14)

In [ ]:
plt.scatter(results_df["% missed contaminants"], results_df["pct_items_inspected_completion"]*100)
plt.title("Missed contaminants vs. items inspected", fontsize=18)
plt.ylabel("Pct. missed contaminants", fontsize=14)
plt.xlabel("Pct. items inspected", fontsize=14)